##### Bibliotecas

In [37]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
import re
import time

##### Navegador

In [38]:
nav = webdriver.Chrome()

##### Base de dados

In [39]:
tabela_produtos = pd.read_excel('buscas.xlsx')
display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64 gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


##### Google Shopping

- Variáveis

In [40]:
produto = "iphone 12 64 gb"
termos_banidos = 'mini watch'
preco_minimo = 3000
preco_maximo = 3500

- Função

In [41]:
def busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo):

  produto = produto.lower()
  termos_banidos = termos_banidos.lower()
  lista_termos_banidos = termos_banidos.split(' ')
  lista_termos_nome_produto = produto.split(' ')
  lista_ofertas = []
  
  preco_minimo = float(preco_minimo)
  preco_maximo = float(preco_maximo)

  nav.get('https://www.google.com.br/')

  nav.find_element(By.XPATH, '//*[@id="APjFqb"]').send_keys(produto, Keys.ENTER)

  # Aba Shopping
  elementos = nav.find_element(By.CLASS_NAME, 'O3S9Rb').click()

  # Preços
  lista_resultados = nav.find_elements(By.CLASS_NAME, 'KZmu8e')

  for resultado in lista_resultados:
    nome = resultado.find_element(By.CLASS_NAME, 'sh-np__product-title').text
    nome = nome.lower()

    tem_termos_banidos = False
    for palavra in termos_banidos.split(' '):
      if palavra in nome:
        tem_termos_banidos = True

    tem_todos_termos_produtos = True
    for palavra in lista_termos_nome_produto:
      if palavra not in nome:
        tem_todos_termos_produtos = False   

    if not tem_termos_banidos and tem_todos_termos_produtos:    
      preco = resultado.find_element(By.CLASS_NAME, 'hn9kf').text
      preco = re.search(r'\d+\.?\d+', preco).group()
      preco = preco.replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.')
      preco = float(preco)

      if preco_minimo <= preco <= preco_maximo:
        elemento_referencia = resultado.find_element(By.CLASS_NAME, 'SirUVb')
        elemento_pai = elemento_referencia.find_element(By.XPATH, './..')
        link = elemento_pai.get_attribute('href')
        lista_ofertas.append((nome, preco, link))
  return lista_ofertas


def busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo):

  produto = produto.lower()
  termos_banidos = termos_banidos.lower()
  lista_termos_banidos = termos_banidos.split(' ')
  lista_termos_nome_produto = produto.split(' ')
  lista_ofertas = []
  preco_minimo = float(preco_minimo)
  preco_maximo = float(preco_maximo)

  nav.get('https://www.buscape.com.br/')
  nav.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto, Keys.ENTER)

  time.sleep(5)
    
  lista_resultados = nav.find_elements(By.CLASS_NAME, 'SearchCard_ProductCard_Inner__7JhKb')

  for resultado in lista_resultados:
    preco = resultado.find_element(By.CLASS_NAME, 'Text_MobileHeadingS__Zxam2').text
    nome = resultado.find_element(By.CLASS_NAME, 'SearchCard_ProductCard_Name__ZaO5o').text
    link = resultado.get_attribute('href')
    print(nome, preco, link)

In [42]:
# lista_ofertas_google_shopping = busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo)
# print(lista_ofertas_google_shopping)

busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo)

Smartphone Apple iPhone 12 64GB Câmera Dupla R$ 3.020,00 https://www.buscape.com.br/celular/smartphone-apple-iphone-12-64gb-ios?_lc=88&searchterm=iphone%2012%2064%20gb
Smartphone Apple iPhone 12 Usado 64GB Câmera Dupla R$ 2.749,00 https://www.buscape.com.br/celular/smartphone-apple-iphone-12-usado-64gb-camera-dupla?_lc=88&searchterm=iphone%2012%2064%20gb
Smartphone Apple iPhone 12 Mini 64GB Câmera Dupla R$ 6.419,00 https://www.buscape.com.br/celular/smartphone-apple-iphone-12-mini-64gb-ios-camera-dupla?_lc=88&searchterm=iphone%2012%2064%20gb
Smartphone Apple iPhone 12 Vermelho 64GB Câmera Dupla R$ 3.779,10 https://www.buscape.com.br/celular/smartphone-apple-iphone-12-vermelho-64gb-ios?_lc=88&searchterm=iphone%2012%2064%20gb
Smartphone Apple iPhone 12 Mini Vermelho 64GB Câmera Dupla R$ 3.779,10 https://www.buscape.com.br/celular/smartphone-apple-iphone-12-mini-vermelho-64gb-ios?_lc=88&searchterm=iphone%2012%2064%20gb
